In [1]:
# Quick and dirty hack because my PyCharm interpreter is not playing nice
ewouts_broken_PyCharm = True
if ewouts_broken_PyCharm:
    import sys
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310')
    sys.path.append('C:\\Users\\Ewout\\Documents\\python_venv\\Py310\\lib\\site-packages')

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import sys
import os
import itertools
import pandas as pd
import pickle
from numpy import random
from time import sleep

### Scaper
https://www.msc.com/en/search-a-schedule

In [3]:
# Create list of origin and destination ports
o_ports = ['BRFOR', 'BRSSZ', 'BRPEC', 'BRSSA', 'BRPNG', 'BRRIO', 'BRRIG', 'BRITJ', 'BRNAT', 'COCTG', 'COSMR', 'COTRB', 'SRPBM', 'CWWIL', 'UYMVD', 'ARBUE', 'CLCNL', 'CLSAI', 'CLVAP', 'PEPAI', 'PECLL', 'ECGYE', 'ECPBO', 'ECPSJ', 'VNVUT', 'VNCMT']
d_ports = ['NLRTM', 'BEANR', 'NLVLI', 'BEZEE', 'NLAMS']

# Make list with all combinations
od_ports = list(itertools.product(o_ports, d_ports))
print(f"{len(od_ports)} combinations of ports")

# Define the base URL to search
base_url = "https://www.msc.com/en/search-a-schedule"

130 combinations of ports


In [4]:
headless = False

# Instantiate options
opts = Options()
#opts.binary_location = "/usr/bin/google-chrome"
opts.binary_location = "C:\Program Files\Google\Chrome\Application\chrome.exe"
if headless:
    opts.headless = True
opts.add_argument('--remote-debugging-port=9222')
opts.add_argument('--disable-gpu')
opts.add_argument("--window-size=1920,1440")

# Some options to make Chrome (hopefully) more
opts.add_argument('--disable-blink-features=AutomationControlled')
opts.add_experimental_option('useAutomationExtension', False)
opts.add_experimental_option("excludeSwitches", ["enable-automation"])

# Set the location of the webdriver
os.environ["WDM_PROGRESS_BAR"] = '0'
chrome_service = Service(ChromeDriverManager().install())

# Instantiate a webdriver
driver = webdriver.Chrome(options=opts, service=chrome_service)

In [5]:
# Get a single pair
o, d = od_ports[0]
o = "BRSSZ"
direct = True

# Load the HTML page
driver.get(base_url)

# Accept cookies
driver.implicitly_wait(3)
try:
    driver.find_element(By.ID,"onetrust-accept-btn-handler").click()
except:
    print("Cookies already accepted.")

# Fill in and select the origin and destination port
for label, port in [("from", o), ("to", d)]:
    elem = driver.find_element(By.ID, label)
    elem.send_keys(port)
    sleep(0.5) ### TODO make this a proper wait
    ActionChains(driver).send_keys(Keys.TAB).send_keys(Keys.ENTER).perform()

# Select direct routing
if direct:
    driver.find_element(By.ID, "directrouting").click()

sleep(1)
# Click search button
driver.find_element(By.CSS_SELECTOR, "button.msc-cta").click()

# Find all buttons to open
buttons = driver.find_elements(By.CSS_SELECTOR, "button.msc-search-schedule-result-details__more-button")

# Open the buttons that are displayed
for btn in buttons:
    if btn.is_displayed():
        btn.click()

# Save soup
#sleep(1) ### TODO proper wait
soup = BeautifulSoup(driver.page_source, features="html.parser")

In [6]:
soup = BeautifulSoup(driver.page_source, features="html.parser")

### Soup
Process the soup

#### Notes
 - It looks like the first two connections are empty (9 connections in the list, 7 on the website, first 2 contain no data)
  - Departure time has multiple fields, the second/last one contains the actual time. Therefore the [1] on the end.

In [56]:
# Split the soup in different connections
test_connections = soup.find_all("div", {"class": "msc-search-schedule-result-details__result"})

In [57]:
if False:
    print(connections[4].prettify())

In [87]:
# Returns connection data for input soups
def get_connection_data(soups):
    # Create empty list for the connection data
    connection_data = []

    # Split each soup in different connections
    for soup1 in soups:
        connections = soup1.find_all("div", {"class": "msc-search-schedule-result-details__result"})

        # For each connection, find data
        for connection in connections:
            transit_time = connection.find("span", {"x-text": "entry.TotalTransitTime"}).text
            co2_footprint = connection.find("span", {"x-text": "entry.CO2FootPrint"}).text

            departure_time = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'EstimatedDepartureTimeFormatted')"})[-1].text
            departure_port = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'DeparturePortName')"})[-1].text
            departure_terminal = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'DepartureEquipmentHandlingFacilityName')"})[-1].text

            service_name = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'MaritimeServiceName')"})[-1].text
            vessel_name = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'Vessel').VesselName"})[-1].text
            "getLegSequenceValue(entry, 'Vessel').VesselName"
            voyage_no = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'DepartureVoyageNo')"})[-1].text

            vessel_build_year = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'Vessel').VesselBuiltYear"})[-1].text
            vessel_flag = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'Vessel').VesselFlag"})[-1].text
            vessel_imo_code =  connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'Vessel').VesselImoCode"})[-1].text

            arrival_port = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'ArrivalPortName')"})[-1].text
            arrival_terminal = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'ArrivalEquipmentHandlingFacilityName')"})[-1].text
            arrival_time = connection.find_all("span", {"x-text": "getLegSequenceValue(entry, 'EstimatedArrivalTimeFormatted')"})[-1].text

            new_connection_data = [departure_port, departure_terminal, arrival_port, arrival_terminal, departure_time, arrival_time, transit_time, service_name, co2_footprint, vessel_name, voyage_no, vessel_build_year, vessel_flag, vessel_imo_code]
            connection_data.append(new_connection_data)
    return connection_data

In [90]:
test_data = get_connection_data([soup])
print(test_data[3:5])

[['SANTOS', 'BRASIL TERMINAL PORTUARIO S/A', 'ROTTERDAM', 'ECT DELTA MARINE TERMINAL (DMT)', 'Fri 4th Nov 2022 08:00', 'Wed 23rd Nov 2022 15:00', '19 days', 'NWC TO SAEC - STRING I', '0.7926 Tons', 'MSC SOFIA CELESTE', 'NA243R', '2015', 'PORTUGAL', '9702091'], ['SANTOS', 'BRASIL TERMINAL PORTUARIO S/A', 'ROTTERDAM', 'ECT DELTA MARINE TERMINAL (DMT)', 'Sun 13th Nov 2022 01:00', 'Wed 30th Nov 2022 15:00', '17 days', 'NWC TO SAEC - STRING I', '0.7926 Tons', 'CZECH', 'NA244R', '2015', 'MALTA', '9723241']]


In [91]:
columns = ["Departure port", "Departure terminal", "Arrival port", "Arrival terminal", "Departure time", "Arrival time", "Transit time", "Service name", "CO2 footprint", "Vessel name", "Voyage no.", "Vessel build year", "Vessel flag", "Vessel imo code"]

In [92]:
df = pd.DataFrame(test_data, columns=columns)
df

,Departure port,Departure terminal,Arrival port,Arrival terminal,Departure time,Arrival time,Transit time,Service name,CO2 footprint,Vessel name,Voyage no.,Vessel build year,Vessel flag,Vessel imo code
0,,,,,,,,,,,,,,
1,,,,,,,,,,,,,,
2,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 28th Oct 2022 02:30,Wed 16th Nov 2022 09:30,19 days,NWC TO SAEC - STRING I,0.7926 Tons,MEHUIN,NA242R,2011,LIBERIA,9400100
3,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 4th Nov 2022 08:00,Wed 23rd Nov 2022 15:00,19 days,NWC TO SAEC - STRING I,0.7926 Tons,MSC SOFIA CELESTE,NA243R,2015,PORTUGAL,9702091
4,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Sun 13th Nov 2022 01:00,Wed 30th Nov 2022 15:00,17 days,NWC TO SAEC - STRING I,0.7926 Tons,CZECH,NA244R,2015,MALTA,9723241
5,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 18th Nov 2022 08:00,Wed 7th Dec 2022 15:00,19 days,NWC TO SAEC - STRING I,0.7926 Tons,MSC ATHENS,NA245R,2013,MALTA,9618305
6,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 25th Nov 2022 08:00,Wed 14th Dec 2022 15:00,19 days,NWC TO SAEC - STRING I,0.7926 Tons,MSC PALAK,NA246R,2016,PORTUGAL,9735206
7,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 2nd Dec 2022 08:00,Wed 21st Dec 2022 15:00,19 days,NWC TO SAEC - STRING I,0.7926 Tons,HUNGARY,NA247R,2015,MALTA,9723253
8,SANTOS,BRASIL TERMINAL PORTUARIO S/A,ROTTERDAM,ECT DELTA MARINE TERMINAL (DMT),Fri 9th Dec 2022 08:00,Wed 28th Dec 2022 15:00,19 days,NWC TO SAEC - STRING I,0.7926 Tons,MSC CATERINA,NA248R,2015,PORTUGAL,9705005
